In [34]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import LineString
import numpy as np

# Wild and Scenic Designations – New England 

In [45]:
# manually curated list from the legislation
points = pd.read_csv("./wspolys/Dingell Bill WSRs - Sheet1.csv")

In [46]:
# create lines from start/end points. Have to split "Lat, Lon" strings into [float(Lon), float(Lat)] for start + end
lines = [LineString([np.array(s[::-1]).astype('float'),np.array(e[::-1]).astype('float')])
         for s, e in zip(points.Start.str.split(','), points.End.str.split(','))]

In [47]:
river_protos = gpd.GeoDataFrame(points, geometry = lines)
river_protos.to_file("./wspolys/northeast-lines.geojson", driver="GeoJSON")
river_protos.crs = {'init' : 'epsg:4326'}


which nhds?

In [6]:
HUC4_subregions = gpd.read_file("NHD/WBD_01_Shape/Shape/WBDHU4.shp")
river_polys_huc4 = gpd.sjoin(river_protos, HUC4_subregions, how='left', op='intersects')
river_polys_huc4.HUC4.unique()

/Users/tony/miniconda3/envs/wildscenic/lib/python3.5/site-packages/geopandas/tools/sjoin.py:44: UserWarning: CRS of frames being joined does not match!
  warn('CRS of frames being joined does not match!')


array(['0108', '0109', '0110', '0107'], dtype=object)

In [7]:
# NHD
! ls NHD/

0107_Shape               1807_Shape               WBD18_HU2_Shape
0108_Shape               1809_Shape               WBD_01_HU2_Shape.zip
0109_Shape               1810_Shape               WBD_01_Shape
0110_Shape               NHD_H_0107_HU4_Shape.zip
1806_Shape               NHD_H_0108_HU4_Shape.zip


In [30]:
nhd_0107 = gpd.read_file("NHD/0107_Shape/Shape/NHDFlowline.shp")
nhd_0108 = gpd.read_file("NHD/0108_Shape/Shape/NHDFlowline.shp")
nhd_0109 = gpd.read_file("NHD/0109_Shape/Shape/NHDFlowline.shp")
nhd_0110 = gpd.read_file("NHD/0110_Shape/Shape/NHDFlowline.shp")


nhd = gpd.GeoDataFrame(pd.concat([nhd_0107, nhd_0108, nhd_0109, nhd_0110]), geometry='geometry')
nhd.crs = nhd_0109.crs

**join based on name, then crop based on buffered line envelope**

In [52]:
# group NHD flowlines by name.
nhd_dissolve = nhd.dissolve(by="GNIS_Name", aggfunc='sum').reset_index()

In [48]:
merged = river_protos.merge(nhd_dissolve, left_on = 'River Name', right_on='GNIS_Name')

In [49]:
final_geoms = [ i.geometry_y.intersection(i.geometry_x.envelope.buffer(0.1))for _, i in merged.iterrows()]

In [50]:
finals = gpd.GeoDataFrame(merged, geometry = final_geoms, crs = nhd_dissolve.crs).drop(columns=['geometry_x', 'geometry_y'])
finals.to_file("northeast-v3.geojson", driver='GeoJSON')

In [33]:
gpd.GeoDataFrame(geometry=[i.envelope for i in merged.geometry_x], crs=nhd.crs).to_file("envelopes.geojson", driver='GeoJSON')

In [37]:
nhd_dissolve.to_file("nhd_all")